# II) NLP Data Processing

## 1) Ingredients

This part can be challenging, this first thing is that they are a lot of different words and some can be useless for our machine learning algortihm. The other thing is that with the data imported from the web scraping the words can be slightly different, so they won't match with our data set.

For the first step, we import the data and keep only the features of interest. They are many features in the dataset that we won't use and the values 'Sauces condimentaires_RHF' and 'Bouillons et potages_RHF' are old data we should remove.

We can see on the DataFrame that they are 4 features.
The feature called Ingrédient contains too many informations, we want to have a clean "Ingrédient" without too many possibilities in order to match easily with the data from the web-scraping.
1. We will delete every ingredients after the ":" which describes us the composition of this ingredient. We can give one exemple with "chocolat noir : masse de cacao, sucre, emulsifiant : lecithine de soja"
We will do this for 3 reasons, the first one is that some data from the web-scraping won't put every detail about the ingredient but just "chocolat noir" because the data from Oqali are really precise. The second is that they are not the main composition of the product, they are just a part of an ingredient, we can suppose that in the majority their % in the product is really low. Finally we tested different classification with them and without them and the result where almost the same.
+ add that they will introduce the unicity of the ingredient
+ they are "conservateur", "epaississant", ect ... not really important and they composition can vary from a brand to another
+ new shop don't take them into account, won't work
+ difficult to automate this
+ underfitting, over fitting
2. We will delete every ingredients that are in "()" for the same reasons as ":"
3. To see if the data is clean and for the rest of the study we will remplace every " " with "_"
4. We drop every rows that contain an ingredient which is unique in the data

In [8]:
import pandas as pd
import numpy as np

df = pd.read_csv(r'C:\Users\Thomas Aujoux\Documents\GitHub\food-classification\data_preparation\data_oqali\ingredients.csv', low_memory=False, sep=';')
df = df[["Code produit", "Secteur", "Famille", "Ingrédient"]]
df = df.rename(columns={ 'Code produit': 'Code_produit' })
df = df[~df['Secteur'].isin(['Sauces condimentaires_RHF', 'Bouillons et potages_RHF'])]
df.head()

,Code_produit,Secteur,Famille,Ingrédient
0,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,lait ecreme reconstitue
1,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,sucre
2,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,fruit : framboise
3,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,creme
4,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,"epaississants : amidon transforme de mais, gom..."


In [10]:
def remove_colon(list):
    n = len(list)
    i = 1
    colonfree = list[0]
    while i < n and list[i] != ":":
        colonfree = colonfree + " " + list[i]
        i = i + 1
    return colonfree

df.Ingrédient[df.Ingrédient.str.contains(":")] = df[df.Ingrédient.str.contains(":")].Ingrédient.str.split().map(lambda x:remove_colon(x))

df.Ingrédient[df.Ingrédient.str.contains("\(")] = df[df.Ingrédient.str.contains("\(")].Ingrédient.str.replace(r"\s\(.*\)", "", regex=True)
df.Ingrédient[df.Ingrédient.str.contains("\(")] = df[df.Ingrédient.str.contains("\(")].Ingrédient.str.replace(r"\s\(.*\)\s", " ", regex=True)
df.Ingrédient[df.Ingrédient.str.contains("\(")] = df[df.Ingrédient.str.contains("\(")].Ingrédient.str.replace(r"\(.*\)", "", regex=True)
df.Ingrédient[df.Ingrédient.str.contains("\(")] = df[df.Ingrédient.str.contains("\(")].Ingrédient.str.replace(r"\(.*", "", regex=True)
df.Ingrédient = df["Ingrédient"].apply(lambda x: x.rstrip())
df.Ingrédient = df["Ingrédient"].apply(lambda x: x.lstrip())
df = df[df.duplicated(subset=['Ingrédient'], keep=False)]
df['Ingrédient'] = df['Ingrédient'].apply(lambda x : x.replace('_',' '))
df.head()

C:\Users\Thomas Aujoux\AppData\Local\Temp\ipykernel_20744\667864613.py:11: ChainedAssignmentError: A value is trying to be set on a copy of a DataFrame or Series through chained assignment.
When using the Copy-on-Write mode, such chained assignment never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy.

Try using '.loc[row_indexer, col_indexer] = value' instead, to perform the assignment in a single step.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Ingrédient[df.Ingrédient.str.contains(":")] = df[df.Ingrédient.str.contains(":")].Ingrédient.str.split().map(lambda x:remove_colon(x))
C:\Users\Thomas Aujoux\AppData\Local\Temp\ipykernel_20744\667864613.py:13: ChainedAssignmentError: A value is trying to be set on a copy of a DataFrame or Series through chained assignment.
When using the Copy-on-Write mo

,Code_produit,Secteur,Famille,Ingrédient
0,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,lait ecreme reconstitue
1,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,sucre
2,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,fruit
3,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,creme
4,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,epaississants


In [ ]:
df.loc[df["Ingrédient"].str.contains(":")] = df[df.Ingrédient.str.contains(":")].Ingrédient.str.split().map(lambda x:remove_colon(x))

In [33]:
df.head()

,Code_produit,Secteur,Famille,Ingrédient
0,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,lait ecreme reconstitue
1,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,sucre
2,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,fruit
3,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,creme
4,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,epaississants


In [54]:
import string
string.punctuation
list_punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890'

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~12345678901234567890123456789012345678901234567890'

In [34]:
#library that contains punctuation
import string
string.punctuation
string.punctuation = string.punctuation + '1234567890'
list_punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890'
#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in list_punctuation])
    return punctuationfree

#storing the puntuation free text
df['clean_msg']= df['Ingrédient'].apply(lambda x:remove_punctuation(x))
df.head()

,Code_produit,Secteur,Famille,Ingrédient,clean_msg
0,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,lait ecreme reconstitue,lait ecreme reconstitue
1,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,sucre,sucre
2,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,fruit,fruit
3,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,creme,creme
4,450,Produits laitiers et desserts frais,Yaourts et laits fermentes sucres classiques,epaississants,epaississants


In [35]:
df['msg_lower']= df['clean_msg'].apply(lambda x: x.lower())
df = df.groupby(['Code_produit', 'Secteur'])['msg_lower'].agg(lambda col: ' '.join(col)).reset_index(name='msg_lower')

In [63]:
df.head()

,Code_produit,Secteur,msg_lower,msg_tokenied
0,450,Produits laitiers et desserts frais,lait ecreme reconstitue sucre fruit creme epai...,"[lait, ecreme, reconstitue, sucre, fruit, crem..."
1,453,Produits laitiers et desserts frais,yaourt au lait partiellement ecreme sucre siro...,"[yaourt, au, lait, partiellement, ecreme, sucr..."
2,455,Produits laitiers et desserts frais,yaourt au lait mg sucre fruits fruit fruit fr...,"[yaourt, au, lait, , mg, sucre, fruits, fruit,..."
3,456,Produits laitiers et desserts frais,yaourt au lait mg sucre fraise fructose conce...,"[yaourt, au, lait, , mg, sucre, fraise, fructo..."
4,460,Produits laitiers et desserts frais,lait ecreme pasteurise creme pasteurisee ferme...,"[lait, ecreme, pasteurise, creme, pasteurisee,..."


In [36]:
#defining function for tokenization
import re
def tokenization(text):
    tokens = text.split(sep = ' ')
    return tokens
#applying function to the column
df['msg_tokenied']= df['msg_lower'].apply(lambda x: tokenization(x))

In [46]:
text = 'lait ecreme reconstitue sucre fruit creme epaississants epaississant epaississant colorant mineraux du lait jus concentre de sureau aromes ferments lactiques du yaourt vitamine b vitamine b vitamine d'
tokens = re.split('W+',text)

In [64]:
df.head()

,Code_produit,Secteur,msg_lower,msg_tokenied
0,450,Produits laitiers et desserts frais,lait ecreme reconstitue sucre fruit creme epai...,"[lait, ecreme, reconstitue, sucre, fruit, crem..."
1,453,Produits laitiers et desserts frais,yaourt au lait partiellement ecreme sucre siro...,"[yaourt, au, lait, partiellement, ecreme, sucr..."
2,455,Produits laitiers et desserts frais,yaourt au lait mg sucre fruits fruit fruit fr...,"[yaourt, au, lait, , mg, sucre, fruits, fruit,..."
3,456,Produits laitiers et desserts frais,yaourt au lait mg sucre fraise fructose conce...,"[yaourt, au, lait, , mg, sucre, fraise, fructo..."
4,460,Produits laitiers et desserts frais,lait ecreme pasteurise creme pasteurisee ferme...,"[lait, ecreme, pasteurise, creme, pasteurisee,..."


In [40]:
#importing nlp library
import nltk
#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('french')
stopwords.append("a")
stopwords.append("mg")
stopwords

['au',
 'aux',
 'avec',
 'ce',
 'ces',
 'dans',
 'de',
 'des',
 'du',
 'elle',
 'en',
 'et',
 'eux',
 'il',
 'ils',
 'je',
 'la',
 'le',
 'les',
 'leur',
 'lui',
 'ma',
 'mais',
 'me',
 'même',
 'mes',
 'moi',
 'mon',
 'ne',
 'nos',
 'notre',
 'nous',
 'on',
 'ou',
 'par',
 'pas',
 'pour',
 'qu',
 'que',
 'qui',
 'sa',
 'se',
 'ses',
 'son',
 'sur',
 'ta',
 'te',
 'tes',
 'toi',
 'ton',
 'tu',
 'un',
 'une',
 'vos',
 'votre',
 'vous',
 'c',
 'd',
 'j',
 'l',
 'à',
 'm',
 'n',
 's',
 't',
 'y',
 'été',
 'étée',
 'étées',
 'étés',
 'étant',
 'étante',
 'étants',
 'étantes',
 'suis',
 'es',
 'est',
 'sommes',
 'êtes',
 'sont',
 'serai',
 'seras',
 'sera',
 'serons',
 'serez',
 'seront',
 'serais',
 'serait',
 'serions',
 'seriez',
 'seraient',
 'étais',
 'était',
 'étions',
 'étiez',
 'étaient',
 'fus',
 'fut',
 'fûmes',
 'fûtes',
 'furent',
 'sois',
 'soit',
 'soyons',
 'soyez',
 'soient',
 'fusse',
 'fusses',
 'fût',
 'fussions',
 'fussiez',
 'fussent',
 'ayant',
 'ayante',
 'ayantes',


In [41]:


#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

#applying the function
df['no_stopwords']= df['msg_tokenied'].apply(lambda x:remove_stopwords(x))

In [42]:
import nltk
import string
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to C:\Users\Thomas
[nltk_data]     Aujoux\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Thomas
[nltk_data]     Aujoux\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Thomas
[nltk_data]     Aujoux\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [69]:

lemmatizer = FrenchLefffLemmatizer()

def preprocess_text(test):
    a = len(df)
    for i in range(len(df)):
        for j in range(len(df.iloc[i, 2])):
            df.iloc[i, 2][j] = lemmatizer.lemmatize(df.iloc[i, 2][j])
            if len(df.iloc[i, 2][j])<=2 :
                df.iloc[i, 2][j] = ""
            if len(df.iloc[i, 2][j])>3 and df.iloc[i, 2][j][-3 :] == "ees":
                df.iloc[i, 2][j] = df.iloc[i, 2][j][:-2]
            if len(df.iloc[i, 2][j])>3 and df.iloc[i, 2][j][-1] == "s":
                df.iloc[i, 2][j] = df.iloc[i, 2][j][:-1]
            if len(df.iloc[i, 2][j])>3 and df.iloc[i, 2][j][-2 :] == "ee":
                df.iloc[i, 2][j] = df.iloc[i, 2][j][:-1]
    return df
            


In [68]:
len("dd")

2

In [117]:
lemmatizer.lemmatize("fruits")

'fruit'

In [70]:
df = preprocess_text(df)

In [105]:
df.iloc[2,2][1]

'lait'

In [55]:
lemmatizer.lemmatize("noire")

'noire'

In [50]:
df = df[["Code_produit", "Secteur", "no_stopwords"]]

In [71]:
from sklearn.feature_extraction.text import CountVectorizer
coun_vect = CountVectorizer(analyzer=lambda x: x)
count_matrix = coun_vect.fit_transform(df["no_stopwords"])
count_array = count_matrix.toarray()
df_vect = pd.DataFrame(data=count_array,columns = coun_vect.get_feature_names_out())
new_df = pd.concat([df[["Code_produit", "Secteur"]], df_vect], axis=1)

In [72]:
a = new_df.columns.values

In [75]:
new_df.to_csv(r'C:\Users\Thomas Aujoux\Documents\GitHub\food-classification\data_preparation\clean.csv')

In [64]:
f = "sdfdsfsd"
f[-2 : ]

'sd'